## Functions

In [1]:

from LyndonWords import *
import itertools
import progressbar
import math
import numpy as np
import cProfile
def genPermutations(type,n,k):
    rootSystemPermutations = np.empty(math.factorial(n+1),dtype=object)
    bar = progressbar.ProgressBar(maxval=math.factorial(n+1))
    v = 0
    for i in bar(itertools.permutations(list(range(n+1)))):
        rootSystemPermutations[v] = rootSystem(i,type,k)
        v+=1
    return rootSystemPermutations

In [2]:
def checkMonotoneIncreasingImpDeltaStart(rootSystemPermutations):
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.getWords(rootsys.delta)
        temp = rootsys.checkMonotonicity("Increasing")
        monotoneIncreasing = [rootsys.getWords(i[0] + rootsys.delta)[0] for i in temp]
        deltaArrays = [i.string for i in deltaWords]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(str(i).startswith(str(j))):
                    flag = True
                    break
            if(flag):
                continue
            for j in rootsys.getAffineWords(i.weights):
                if np.any(np.all(j[0:sum(rootsys.delta)] == deltaArrays)):
                    returnArr.append((rootsys.ordering,j))
    return returnArr

In [3]:
def checkDeltaStartImpMonotoneIncreasing(rootSystemPermutations):
    returnArr = []
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.getWords(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.getWords(base + rootsys.delta)[0]
            flag = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaDegree] == i)):
                    flag = True
                    break
            if not flag:
                continue
            if(rootsys.getMonotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.getWords(base)[0])))
    return returnArr

In [4]:
def printByLength(rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.getWords(comb):
                print(" "+str(i))

In [5]:
def printByBaseWord(rootSystem:rootSystem,comb):
    k=0
    isImaginary = rootSystem.isImaginary(comb)
    for i in rootSystem.getAffineWords(comb):
        if(isImaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not isImaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [6]:
def exportByLength(rootSystem):
    file = open("out.txt",'w')
    file.write(str(rootSystem.ordering)+"\n")
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            for i in rootSystem.getWords(comb):
                file.write(str(i) + '\n')
    file.close()

In [7]:
def checkDeltaPattern(perms,k=2,printDelta = False):
    for i in perms:
        deltaWords = i.getWords(k*i.delta)
        flag = True
        for j in range(1,len(deltaWords)):
            if(deltaWords[j][sum(i.delta)-1] != deltaWords[j][0]):
                if(flag):
                    print(i.ordering)
                    flag = False
                if(printDelta):
                    print(f"  SL_{j}(\delta): " + str(i.getWords(i.delta)[j]))
                print(f"  SL_{j}({k}*\delta): " + str(deltaWords[j]))

In [8]:
def checkHDeltasRepeat(perms):
    excepts = []
    for i in perms:
        hs = [j.hs for j in i.getAffineWords(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

## Type C

In [83]:
Cperms = genPermutations('C',4,3)

  0% (0 of 120) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  1% (2 of 120) |                        | Elapsed Time: 0:00:00 ETA:   0:00:09
  3% (4 of 120) |                        | Elapsed Time: 0:00:00 ETA:   0:00:09
  4% (5 of 120) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:09
  5% (7 of 120) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:08
  6% (8 of 120) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:08
  8% (10 of 120) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:08
  9% (11 of 120) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:08
 10% (13 of 120) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:08
 11% (14 of 120) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:08
 13% (16 of 120) |###                    | Elapsed Time: 0:00:01 ETA:   0:00:08
 14% (17 of 120) |###                    | Elapsed Time: 0:00:01 ETA:   0:00:08
 15% (19 of 120) |###                   

In [84]:
exceptions = checkMonotoneIncreasingImpDeltaStart(Cperms)
if(len(exceptions) != 0):
    print("Exception")

Above shows that monotone increasing implies it starts with a delta word

In [85]:
checkDeltaStartImpMonotoneIncreasing(Cperms)

[('0<1<2<3<4', '1'),
 ('0<1<2<3<4', '2'),
 ('0<1<2<3<4', '3'),
 ('0<1<2<3<4', '4'),
 ('0<1<2<3<4', '3,4'),
 ('0<1<2<3<4', '1,2'),
 ('0<1<2<3<4', '2,3'),
 ('0<1<2<3<4', '2,3,4'),
 ('0<1<2<3<4', '1,2,3'),
 ('0<1<2<3<4', '3,3,4'),
 ('0<1<2<3<4', '1,2,3,4'),
 ('0<1<2<3<4', '2,3,4,3'),
 ('0<1<2<3<4', '1,2,3,4,3'),
 ('0<1<2<3<4', '2,3,2,3,4'),
 ('0<1<2<3<4', '1,2,3,4,3,2'),
 ('0<1<2<3<4', '1,2,3,1,2,3,4'),
 ('0<1<2<4<3', '1'),
 ('0<1<2<4<3', '2'),
 ('0<1<2<4<3', '3'),
 ('0<1<2<4<3', '4'),
 ('0<1<2<4<3', '4,3'),
 ('0<1<2<4<3', '1,2'),
 ('0<1<2<4<3', '2,3'),
 ('0<1<2<4<3', '2,3,4'),
 ('0<1<2<4<3', '1,2,3'),
 ('0<1<2<4<3', '4,3,3'),
 ('0<1<2<4<3', '1,2,3,4'),
 ('0<1<2<4<3', '2,3,4,3'),
 ('0<1<2<4<3', '1,2,3,4,3'),
 ('0<1<2<4<3', '2,3,2,3,4'),
 ('0<1<2<4<3', '1,2,3,4,3,2'),
 ('0<1<2<4<3', '1,2,3,1,2,3,4'),
 ('0<1<3<2<4', '1'),
 ('0<1<3<2<4', '2'),
 ('0<1<3<2<4', '3'),
 ('0<1<3<2<4', '4'),
 ('0<1<3<2<4', '3,4'),
 ('0<1<3<2<4', '1,2'),
 ('0<1<3<2<4', '3,2'),
 ('0<1<3<2<4', '3,4,2'),
 ('0<1<3<2<4',

In [86]:
C205134 = rootSystem([2,0,5,1,3,4],'C',5)
for i in C205134.getAffineWords([0,0,1,0,0,0]):
    print(*C205134.costandardFactorization(i),sep='-')


3-None
2,3,1,0,1-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,2,3,4,1,5,0-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,2,3,4,1,5,0,2,3,4,1,2,3,4,1,5,0-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,2,3,4,1,5,0,2,3,4,1,2,3,4,1,5,0,2,3,4,1,2,3,4,1,5,0-2,3,4,5,4,3
2,3,1,0,1,2,3,4,1,2,3,4,1,5,0,2,3,4,1,2,3,4,1,5,0,2,3,4,1,2,3,4,1,5,0,2,3,4,1,2,3,4,1,5,0-2,3,4,5,4,3


Implies that beginning with a k-delta word implies monotone increasing

In [87]:
checkDeltaPattern(Cperms,3,printDelta=True)

2<0<3<1<4
  SL_3(\delta): 2,3,4,3,2,1,0,1
  SL_3(3*\delta): 2,3,4,3,2,1,0,1,2,1,3,0,2,1,3,0,2,1,3,4,2,1,3,4
2<0<3<4<1
  SL_3(\delta): 2,3,4,3,2,1,0,1
  SL_3(3*\delta): 2,3,4,3,2,1,0,1,2,1,3,0,2,1,3,0,2,1,3,4,2,1,3,4
2<0<4<3<1
  SL_3(\delta): 2,3,4,3,2,1,0,1
  SL_3(3*\delta): 2,3,4,3,2,1,0,1,2,1,3,0,2,1,3,0,2,1,3,4,2,1,3,4
2<4<0<3<1
  SL_3(\delta): 2,3,4,3,2,1,0,1
  SL_3(3*\delta): 2,3,4,3,2,1,0,1,2,1,3,2,1,3,0,4,2,1,3,2,1,3,0,4


In [88]:
checkHDeltasRepeat(Cperms)

[]

In [89]:
index=0
[print(*list(Cperms[index].costandardFactorization(i))+[i.hs],sep='-',) for i in Cperms[index].getAffineWords(Cperms[index].delta)]

0,1,2,3,4,3,2-1-[1 0 0 0]
0,1,2,3,4,3-1,2-[1 1 0 0]
0,1,2,3,4-1,2,3-[1 1 1 0]
0-1,2,3,1,2,3,4-[2 2 2 1]
0,1,2,3,4,3,2-0,1,2,3,4,3,2,1,1-[1 0 0 0]
0,1,2,3,4,3,1-0,1,2,3,4,3,2,1,2-[0 1 0 0]
0,1,2,3,4,1,2-0,1,2,3,4,3,1,2,3-[0 0 1 0]
0,1,2,3,1,2,3-0,1,2,3,4,1,2,3,4-[0 0 0 1]
0,1,2,3,4,3,2-0,1,2,3,4,3,2,1,0,1,2,3,4,3,2,1,1-[1 0 0 0]
0,1,2,3,4,3,1-0,1,2,3,4,3,2,1,0,1,2,3,4,3,2,1,2-[0 1 0 0]
0,1,2,3,4,1,2-0,1,2,3,4,3,1,2,0,1,2,3,4,3,1,2,3-[0 0 1 0]
0,1,2,3,1,2,3-0,1,2,3,4,1,2,3,0,1,2,3,4,1,2,3,4-[0 0 0 1]
0,1,2,3,4,3,2-0,1,2,3,4,3,2,1,0,1,2,3,4,3,2,1,0,1,2,3,4,3,2,1,1-[1 0 0 0]
0,1,2,3,4,3,1-0,1,2,3,4,3,2,1,0,1,2,3,4,3,2,1,0,1,2,3,4,3,2,1,2-[0 1 0 0]
0,1,2,3,4,1,2-0,1,2,3,4,3,1,2,0,1,2,3,4,3,1,2,0,1,2,3,4,3,1,2,3-[0 0 1 0]
0,1,2,3,1,2,3-0,1,2,3,4,1,2,3,0,1,2,3,4,1,2,3,0,1,2,3,4,1,2,3,4-[0 0 0 1]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Type G

In [90]:
G2perms = genPermutations('G',2,5)

  0% (0 of 6) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 33% (2 of 6) |########                  | Elapsed Time: 0:00:00 ETA:   0:00:00
 50% (3 of 6) |#############             | Elapsed Time: 0:00:00 ETA:   0:00:00
 66% (4 of 6) |#################         | Elapsed Time: 0:00:00 ETA:   0:00:00
 83% (5 of 6) |#####################     | Elapsed Time: 0:00:00 ETA:   0:00:00
100% (6 of 6) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [91]:
checkDeltaStartImpMonotoneIncreasing(G2perms)

[('0<1<2', '1'),
 ('0<1<2', '2'),
 ('0<1<2', '1,2'),
 ('0<1<2', '1,2,2'),
 ('0<1<2', '1,2,2,2'),
 ('0<1<2', '1,2,1,2,2'),
 ('0<2<1', '1'),
 ('0<2<1', '2'),
 ('0<2<1', '2,1'),
 ('0<2<1', '2,2,1'),
 ('0<2<1', '2,2,2,1'),
 ('0<2<1', '2,2,1,2,1'),
 ('1<0<2', '0'),
 ('1<0<2', '1,2,2'),
 ('1<0<2', '1,2,2,0'),
 ('1<2<0', '2'),
 ('1<2<0', '1,0'),
 ('1<2<0', '1,0,2'),
 ('1<2<0', '1,0,2,2'),
 ('1<2<0', '1,0,2,2,2'),
 ('2<0<1', '0'),
 ('2<0<1', '2,1'),
 ('2<0<1', '2,1,0'),
 ('2<0<1', '2,2,1,0'),
 ('2<0<1', '2,1,2,1,0'),
 ('2<1<0', '0'),
 ('2<1<0', '2,1'),
 ('2<1<0', '2,1,0'),
 ('2<1<0', '2,2,1,0'),
 ('2<1<0', '2,1,2,1,0')]

In [92]:
checkMonotoneIncreasingImpDeltaStart(G2perms)

[]

In [93]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [94]:
index = 4
print(str(G2perms[index].ordering))

printByBaseWord(G2perms[index],G2perms[0].delta)

2<0<1
0
 2,2,1,0,2,1
 2,2,1,2,1,0
1
 2,2,1,2,1,0,2,1,2,2,1,0
 2,2,1,2,1,2,2,1,0,2,1,0
2
 2,2,1,2,1,0,2,2,1,0,2,2,1,2,1,0,2,1
 2,2,1,2,1,2,2,1,0,2,1,2,2,1,0,2,1,0
3
 2,2,1,2,1,0,2,2,1,2,1,0,2,1,2,2,1,2,1,0,2,2,1,0
 2,2,1,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,0
4
 2,2,1,2,1,0,2,2,1,2,1,0,2,2,1,0,2,2,1,2,1,0,2,2,1,2,1,0,2,1
 2,2,1,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,0
5
 2,2,1,2,1,0,2,2,1,2,1,0,2,2,1,2,1,0,2,1,2,2,1,2,1,0,2,2,1,2,1,0,2,2,1,0
 2,2,1,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,2,2,1,0,2,1,0


In [95]:
checkDeltaPattern(G2perms)

In [96]:
checkHDeltasRepeat(G2perms)

[]

## Type B

In [9]:
Bperms = genPermutations('B',5,3)

  0% (0 of 720) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (1 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:11
  0% (2 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:07
  0% (3 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:06
  0% (4 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:04
  0% (5 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:04
  0% (6 of 720) |                        | Elapsed Time: 0:00:01 ETA:   0:02:05
  0% (7 of 720) |                        | Elapsed Time: 0:00:01 ETA:   0:02:05
  1% (8 of 720) |                        | Elapsed Time: 0:00:01 ETA:   0:02:04
  1% (9 of 720) |                        | Elapsed Time: 0:00:01 ETA:   0:02:04
  1% (10 of 720) |                       | Elapsed Time: 0:00:01 ETA:   0:02:03
  1% (11 of 720) |                       | Elapsed Time: 0:00:01 ETA:   0:02:03
  1% (12 of 720) |                      

In [98]:
checkDeltaStartImpMonotoneIncreasing(Bperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '4,5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,4,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '4,5,5'),
 ('0<1<2<3<4<5', '2,3,4,5'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '3,4,5,5'),
 ('0<1<2<3<4<5', '1,2,3,4,5'),
 ('0<1<2<3<4<5', '2,3,4,5,5'),
 ('0<1<2<3<4<5', '3,4,5,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5'),
 ('0<1<2<3<4<5', '2,3,4,5,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4'),
 ('0<1<2<3<4<5', '2,3,4,5,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,4,5,5,4,3,2'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '5,4'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4',

In [99]:
checkMonotoneIncreasingImpDeltaStart(Bperms)

[]

In [100]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [101]:
checkDeltaPattern(Bperms,printDelta=True)

2<0<5<1<3<4
  SL_3(\delta): 2,3,1,0,2,3,4,5,5,4
  SL_3(2*\delta): 2,3,1,0,2,3,4,5,5,0,2,3,4,1,2,3,4,5,5,4
2<0<5<1<4<3
  SL_3(\delta): 2,3,1,0,2,3,4,5,5,4
  SL_3(2*\delta): 2,3,1,0,2,3,4,5,5,0,2,3,4,1,2,3,4,5,5,4
2<1<5<0<3<4
  SL_3(\delta): 2,3,0,1,2,3,4,5,5,4
  SL_3(2*\delta): 2,3,0,1,2,3,4,5,5,1,2,3,4,0,2,3,4,5,5,4
2<1<5<0<4<3
  SL_3(\delta): 2,3,0,1,2,3,4,5,5,4
  SL_3(2*\delta): 2,3,0,1,2,3,4,5,5,1,2,3,4,0,2,3,4,5,5,4
2<4<0<1<3<5
  SL_4(\delta): 2,1,0,2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0,2,3,4,5,5,4,3,2,3,0,4,5,5,2,3,1,4
2<4<0<1<5<3
  SL_4(\delta): 2,1,0,2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0,2,3,4,5,5,4,3,2,3,0,4,5,5,2,3,1,4
2<4<0<5<1<3
  SL_4(\delta): 2,1,0,2,3,4,5,5,4,3
  SL_4(2*\delta): 2,1,0,2,3,4,5,5,4,3,2,3,0,4,5,5,2,3,1,4
2<4<1<0<3<5
  SL_4(\delta): 2,0,1,2,3,4,5,5,4,3
  SL_4(2*\delta): 2,0,1,2,3,4,5,5,4,3,2,3,1,4,5,5,2,3,0,4
2<4<1<0<5<3
  SL_4(\delta): 2,0,1,2,3,4,5,5,4,3
  SL_4(2*\delta): 2,0,1,2,3,4,5,5,4,3,2,3,1,4,5,5,2,3,0,4
2<4<1<5<0<3
  SL_4(\delta): 2,0,1,2,3,4,5,5,4,

In [103]:
checkHDeltasRepeat(Bperms)

[]

## Type D

In [104]:
Dperms = genPermutations('D',5,4)

  0% (0 of 720) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (1 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:05
  0% (2 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:04
  0% (3 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:05
  0% (4 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:05
  0% (5 of 720) |                        | Elapsed Time: 0:00:00 ETA:   0:02:05
  0% (6 of 720) |                        | Elapsed Time: 0:00:01 ETA:   0:02:05
  0% (7 of 720) |                        | Elapsed Time: 0:00:01 ETA:   0:02:04
  1% (8 of 720) |                        | Elapsed Time: 0:00:01 ETA:   0:02:05
  1% (9 of 720) |                        | Elapsed Time: 0:00:01 ETA:   0:02:05
  1% (10 of 720) |                       | Elapsed Time: 0:00:01 ETA:   0:02:04
  1% (11 of 720) |                       | Elapsed Time: 0:00:01 ETA:   0:02:04
  1% (12 of 720) |                      

In [105]:
checkMonotoneIncreasingImpDeltaStart(Dperms)

[]

In [106]:
checkDeltaStartImpMonotoneIncreasing(Dperms)

[('0<1<2<3<4<5', '1'),
 ('0<1<2<3<4<5', '2'),
 ('0<1<2<3<4<5', '3'),
 ('0<1<2<3<4<5', '4'),
 ('0<1<2<3<4<5', '5'),
 ('0<1<2<3<4<5', '1,2'),
 ('0<1<2<3<4<5', '2,3'),
 ('0<1<2<3<4<5', '3,4'),
 ('0<1<2<3<4<5', '3,5'),
 ('0<1<2<3<4<5', '1,2,3'),
 ('0<1<2<3<4<5', '2,3,4'),
 ('0<1<2<3<4<5', '2,3,5'),
 ('0<1<2<3<4<5', '3,5,4'),
 ('0<1<2<3<4<5', '1,2,3,4'),
 ('0<1<2<3<4<5', '1,2,3,5'),
 ('0<1<2<3<4<5', '2,3,5,4'),
 ('0<1<2<3<4<5', '1,2,3,5,4'),
 ('0<1<2<3<4<5', '2,3,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,5,4,3'),
 ('0<1<2<3<4<5', '1,2,3,5,4,3,2'),
 ('0<1<2<3<5<4', '1'),
 ('0<1<2<3<5<4', '2'),
 ('0<1<2<3<5<4', '3'),
 ('0<1<2<3<5<4', '4'),
 ('0<1<2<3<5<4', '5'),
 ('0<1<2<3<5<4', '1,2'),
 ('0<1<2<3<5<4', '2,3'),
 ('0<1<2<3<5<4', '3,4'),
 ('0<1<2<3<5<4', '3,5'),
 ('0<1<2<3<5<4', '1,2,3'),
 ('0<1<2<3<5<4', '2,3,4'),
 ('0<1<2<3<5<4', '2,3,5'),
 ('0<1<2<3<5<4', '3,4,5'),
 ('0<1<2<3<5<4', '1,2,3,4'),
 ('0<1<2<3<5<4', '1,2,3,5'),
 ('0<1<2<3<5<4', '2,3,4,5'),
 ('0<1<2<3<5<4', '1,2,3,4,5'),
 ('0<1<2<3<5<4', '2

In [107]:
checkDeltaPattern(Dperms,2,printDelta=True)

In [108]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [109]:
checkHDeltasRepeat(Bperms)

[]